In [1]:
#!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [2]:
import h2o
from h2o.estimators import (
    H2OGeneralizedLinearEstimator, 
    H2ORandomForestEstimator, 
    H2OGradientBoostingEstimator, 
    H2ONaiveBayesEstimator,
    H2OStackedEnsembleEstimator,
    H2ODeepLearningEstimator

)
from h2o.frame import H2OFrame
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21, mixed mode)
  Starting server from D:\Archivos de programa\Anaconda3\envs\Master_1\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\david\AppData\Local\Temp\tmp73tmtmmz
  JVM stdout: C:\Users\david\AppData\Local\Temp\tmp73tmtmmz\h2o_david_started_from_python.out
  JVM stderr: C:\Users\david\AppData\Local\Temp\tmp73tmtmmz\h2o_david_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,2 months and 4 days
H2O_cluster_name:,H2O_from_python_david_9k1xw8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.979 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


### GLOBAL PRESETS

In [696]:
import warnings
warnings.filterwarnings('ignore')

TEST_SIZE = 0.2

Throughout the project we reference many times the paper: **Practical considerations for specifying a super learner**
https://arxiv.org/pdf/2204.06139

### DATA LOADING AND PREPROCESSING

In [4]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import pandas as pd

In [348]:
spam_data = fetch_openml(data_id=44, as_frame=True)  
spam_df = spam_data.frame
# Split into features and target
X = spam_df.iloc[:, :-1]  # All columns except the last are features
y = spam_df.iloc[:, -1]   # The last column is the target (spam or not)

# Convert target to numeric
y = y.astype(int)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

# Convert to H2O Frames
h2o_train = H2OFrame(pd.DataFrame(X_train).assign(label=y_train.values))
h2o_test = H2OFrame(pd.DataFrame(X_test).assign(label=y_test.values))

h2o_train['label'] = h2o_train['label'].asfactor()
h2o_test['label'] = h2o_test['label'].asfactor()

# Example of dataset
X.head()

C:\Users\vladi\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_%3B,char_freq_%28,char_freq_%5B,char_freq_%21,char_freq_%24,char_freq_%23,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [388]:
X.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_%3B,char_freq_%28,char_freq_%5B,char_freq_%21,char_freq_%24,char_freq_%23,capital_run_length_average,capital_run_length_longest,capital_run_length_total
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.031869,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.285735,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,10.000000,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000


## TODO Try out some feature selection maybe

#### Is SPAM class underepresented?

In [649]:
class_1 = len(spam_df[spam_df['class'] == '1'])
class_0 = len(spam_df[spam_df['class'] == '0'])
print(f"Records containing spam: {class_1}")
print(f"Records not containing spam: {class_0}")

Records containing spam: 1813
Records not containing spam: 2788


#### Computing the effective sample size n_eff (from paper)

We have binary data, the prevalence of Y is **p=class_1 / total_size**, subsequently **n_rare=n*min(p, 1-p)**, and finally **n_eff=min(n, 5*n_rare)** 

In [668]:
n = len(spam_df)

p = class_1 / n
n_rare = n * min(p, 1-p)
n_eff = min(n, 5*n_rare)
n_eff

4601

#### Computing the V for V-fold cross-validation
Since n_eff >= 500 but not >= 5000 we should select a value between 20 and 10. We take in account that n_eff is closer to 5000 and so we focus on V slightly higher than 10.

In [698]:
N_FOLDS = 12

### BASE LEARNERS - SPECIFICATION

When choosing the base learners for the first layer we have considered the properties of the dataset and the task in this case being binary classification. As the paper *Practical considerations for specifying a super learner* suggests "An ideal, rich library is diverse in its learning strategies, able to adapt to a range of underlying functional forms for the true prediction function, computationally feasible, and effective at handling high dimensional data. Diverse libraries include parametric learners, highly data-adaptive learners, multiple variants of the same learner with different tuning parameter specifications...". So the first layer should consist of diverse algorithms with different inductive biases to ensure a rich set of predictions for the metalearnerearner".

We have selected: 

**Random Forest:**

Because they are robust to overfitting on datasets with many features and they handle noisy or irrelevant features well, which is can be a thing in this case.

**Generalized Linear Model - Logistic regression:**

We chose to add it because it's a simple yet effective baseline model, especially logistic regression for binary classification. It should provide a low-variance learner to complement the other high-variance ones.

**Deep Learning (H20's MLP):**

We add the neural networks, because of it's flexibility so it could capture non-linear relationships which should broaden the diversity of the stacks prediction.

**Naive Bayes:**

Why: Spam datasets often benefit from Naive Bayes since it assumes independence among features and thus might capture something more general than the other models.

**Gradient Boosting Machines:**

We choose them as another complement ensemble method that can capture rather complex relationship and so maybe overfit more to th data.


We assume the simpler models like naive bayes and logistic regression should bring in the stack a more general view without focusing too much on the quirks in the data and to balance it out we have selected a more accurate and flexible methods like MLP

Also we try various configurations of hyperparameters for each class of learners as the paper specifies: *"Since the true functional form is unknown, it is a good idea to consider a variety of base learners, and to construct multiple variations of the same base learner with different tuning specifications. There is no harm in including a learner that performs poorly in the library, as it will be given a weight of zero..."* (or close to 0).

In [443]:
base_learners_simple01 = {
    "LogisticRegression": H2OGeneralizedLinearEstimator(family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
    "RandomForest": H2ORandomForestEstimator(ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
    "GradientBoosting": H2OGradientBoostingEstimator(ntrees=50, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True)
}

# TODO consider balance_class=True

base_learners_mix_duplicates01 = {
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_50trees": H2ORandomForestEstimator(
        ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_50trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_10trees": H2ORandomForestEstimator(
        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting": H2OGradientBoostingEstimator(
        ntrees=50, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
    ),
    "NaiveBayes": H2ONaiveBayesEstimator(
        nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "NeuralNetwork_32": H2ODeepLearningEstimator(
        hidden=[32], epochs=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
        hidden=[32, 16], epochs=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    )
}

base_learners_mix_rf_nns_01 = {
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_50trees": H2ORandomForestEstimator(
        ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_50trees_unbounded_D": H2ORandomForestEstimator(
    ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_10trees": H2ORandomForestEstimator(
        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
        ntrees=10, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
    ),
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
        ntrees=50, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
    ),
    "NaiveBayes": H2ONaiveBayesEstimator(
        nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
        hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "NeuralNetwork_32_32": H2ODeepLearningEstimator(
        hidden=[32, 32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "NeuralNetwork_32": H2ODeepLearningEstimator(
        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    )
}

### BASE LEARNERS - TRAINING & EVALUATION

In [394]:
# # Train each base learner using cross-validation
# for name, learner in base_learners.items():
#     print(f"Training {name} with {N_FOLDS}-fold cross-validation...")
#     learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

# results = {}
# for name, learner in base_learners.items():
#     performance = learner.model_performance(test_data=h2o_test)
#     f1_score = performance.F1()[0][1]  
#     auc_pr = performance.aucpr()      
#     accuracy = performance.accuracy()[0][1]
#     results[name] = accuracy
#     results[name] = {"F1-Score": f1_score, "AUC-PR": auc_pr, "Accuracy": accuracy}
#     print(f"{name} - F1-Score: {f1_score:.4f}, AUC-PR: {auc_pr:.4f}, Accuracy (Test Set): {accuracy:.4f}")


# # Print results
# print("Base Learner Results:", results)

# base_models = list(base_learners.values())

In [590]:
def train_evaluate_stack(base_learners, metalearner, h2o_train, h2o_test, X_train):
    
    # TRAIN BASE LEARNERS
    print("\n>>> Training base learners:\n")
    for name, learner in base_learners.items():
        print(f"    Training {name} with {N_FOLDS}-fold cross-validation...")
        learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

    super_learner = H2OStackedEnsembleEstimator(
        base_models=list(base_learners.values()),
        metalearner_algorithm=metalearner
    )
    # TRAIN THE METALEARNER
    print("\n>>> Training super learner:\n")
    super_learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

    # EVAL BASE LEARNERS
    print("\n>>> Base learners' results:\n")
    results = {}
    for name, learner in base_learners.items():
        performance = learner.model_performance(test_data=h2o_test)
        f1_score = performance.F1()[0][1]  
        auc_pr = performance.aucpr()      
        accuracy = performance.accuracy()[0][1]
        results[name] = accuracy
        results[name] = {"F1-Score": f1_score, "AUC-PR": auc_pr, "Accuracy": accuracy}
        print(f"    {name} - F1-Score: {f1_score:.4f}, AUC-PR: {auc_pr:.4f}, Accuracy (Test Set): {accuracy:.4f}")
    
    # EVAL THE METALEARNER
    print("\n>>> Metalearner's results:\n")
    super_performance = super_learner.model_performance(test_data=h2o_test)
    super_accuracy = super_performance.accuracy()[0][1]
    super_f1 = super_performance.F1()[0][1]  
    super_auc_pr = super_performance.aucpr()  
    # print(f"\n    Super Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f} | Super Learner Accuracy: {super_accuracy:.4f}")
    
    
    # print("\nFinal Results Comparison:")
    # for name, metrics in results.items():
    #     print(f"{name} - F1-Score: {metrics['F1-Score']:.4f}, AUC-PR: {metrics['AUC-PR']:.4f}, Accuracy: {metrics['Accuracy']:.4f}")
        
    print(f"    Super Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f}, Accuracy: {super_accuracy:.4f}")
    return {"F1-Score": super_f1, "AUC-PR": super_auc_pr, "Accuracy": super_accuracy}


### METALEARNER - TRAINING & EVALUATION


For the metalearner we principally selected two possible options for testing:

**GLM:**

We chose logistic regression because it is simple and interpretable and as a meta-learner we want it just combine the predictions of individual learners by weighting the them reducing the risk of overfitting when combining predictions. So in this case we are more focused on finding the best combination of predictions rather than adding more complexity.

**Gradient Boosting Machine / MLP:**

As an alternative second option we wanted something stronger, a bit of a bigger hammer sort to say, especially for our stacks which are more diverse in which case their predictions could be more complex, so they could capture non-linear relationships among them.

**Evaluation metrics:**
In accordance with the paper where our task is binary classification of imbalanced classes we chose AUCPR as a primary evaluation metric. In addition as alternatives we provide F1 (once again due uneven class distribution) and finally accuracy as complement.

In [426]:
# # TRAIN
# super_learner = H2OStackedEnsembleEstimator(
#     base_models=base_models,
#     metalearner_algorithm="glm"  # Uses Logistic Regression as the metalearner
# )

# # super_learner = H2OStackedEnsembleEstimator(
# #     base_models=base_models,
# #     metalearner_algorithm="deeplearning"  # Uses Logistic Regression as the metalearner
# # )

# print("\nTraining Super Learner:\n")
# super_learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

# # EVAL
# super_performance = super_learner.model_performance(test_data=h2o_test)
# super_accuracy = super_performance.accuracy()[0][1]
# super_f1 = super_performance.F1()[0][1]  
# super_auc_pr = super_performance.aucpr()  
# # print(f"\nSuper Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f} | Super Learner Accuracy: {super_accuracy:.4f}")


# print("\nFinal Results Comparison:\n")
# for name, metrics in results.items():
#     print(f"    {name} - F1-Score: {metrics['F1-Score']:.4f}, AUC-PR: {metrics['AUC-PR']:.4f}, Accuracy: {metrics['Accuracy']:.4f}")
    
# print(f"    Super Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f}, Accuracy: {super_accuracy:.4f}")




In [700]:
train_evaluate_stack(base_learners_simple01, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training LogisticRegression with 12-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest with 12-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training GradientBoosting with 12-fold cross-validation...
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training NaiveBayes with 12-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    LogisticRegression - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy (Test Set): 0.9251
    RandomForest - F1-Score: 0.9375, AUC-PR: 0.9790, Accuracy (Test Set): 0.9479
    GradientBoosting - 

{'F1-Score': 0.9537275064267353,
 'AUC-PR': 0.98388486064698,
 'Accuracy': 0.9609120521172638}

In [436]:
train_evaluate_stack(base_learners_mix_duplicates01, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_50trees with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_50trees_unbounded_D with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_10trees with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_10trees_unbounded_D with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training NaiveBayes with 5-fold cross-validation...
naivebayes Model Build progress: |████████████████████████████████████████████

In [437]:
train_evaluate_stack(base_learners_mix_rf_nns_01, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_50trees with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_50trees_unbounded_D with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_10trees with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_10trees_unbounded_D with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training GradientBoosting with 5-fold cross-validation...
gbm Model Build progress: |█████████████████████████████████████████████

# Ablation studies

In the following we tried a more methodological way of building the stack. 
We tried two approaches and evaluated their effects on the final test metrics:

**1) Building the stack from simpler models adding more complex ones:**

In this method we start from a base consisting of simple models which we assume would capture the main / most general pattern in the data.
Afterwards we gradually try adding more complex models to extend the stack capabilities to capture more finer intricacies and more complex (perhaps non-linear) relationships in the data and we observe the effect on the test metrics.


**1) Building the stack from more complex models adding more general/simple ones:**
In this method we start from a base consisting of more complex models which we assume would capture the complex relationships in data well and then
we try to bring down the variance by adding simpler models that don't overfit to the data so much.



In [455]:
simple_learners = {
                   "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                   "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True)}


random_forests = {  
                    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
                    "RandomForest_20trees": H2ORandomForestEstimator(
                        ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
                    "RandomForest_50trees": H2ORandomForestEstimator(
                        ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
                    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
                        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
                    "RandomForest_20trees_unbounded_D": H2ORandomForestEstimator(
                        ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
                    "RandomForest_50trees_unbounded_D": H2ORandomForestEstimator(
                        ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    )}

gradient_boostings = {

                    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
                    ),
                    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
                        ntrees=20, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
                    ),
                    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
                        ntrees=50, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
                    ),
                    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
                        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
                    ),
                    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
                        ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
                    ),
                    "GradientBoosting_50trees_unbounded_D": H2OGradientBoostingEstimator(
                        ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True, balance_classes=True
                    )}

neural_networks = {
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32": H2ODeepLearningEstimator(
                        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
                    hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
                    "NeuralNetwork_32_32": H2ODeepLearningEstimator(
                        hidden=[32, 32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
}

### A more efficient variant would be training each model only once in case it is present in multiple combinations.

Due to the tradeoff between the scope of this project and time capabilities we perform only superficial overview. If the problem would be a topic of major research where the time needed to search the vast hypothesis space is available, we would suggest performing more extensive per-class tests with higher hyperparameter sampling granularity to better observe how they affect the models performance.

In [592]:
simple_to_complex01 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}

simple_to_complex02 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "RandomForest_20trees": H2ORandomForestEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex04= {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "RandomForest_20trees": H2ORandomForestEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "RandomForest_50trees": H2ORandomForestEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex05 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "RandomForest_50trees": H2ORandomForestEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex06 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}


simple_to_complex07 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                    ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                ),   
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex08 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex09 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_20trees_unbounded_D": H2ORandomForestEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex10 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_20trees_unbounded_D": H2ORandomForestEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "RandomForest_50trees_unbounded_D": H2ORandomForestEstimator(
    ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

random_forest_stacks = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [594]:
comparative_results = dict()

for i, stack in enumerate(random_forest_stacks):
    comparative_results[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    LogisticRegression_binomial - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy (Test Set): 0.9251

>>> Metalearner's results:

    Super Learner - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy: 0.9251

>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training RandomForest_10trees with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |███████████████████████

In [596]:
comparative_results

{0: {'F1-Score': 0.9125475285171102,
  'AUC-PR': 0.9597218422898272,
  'Accuracy': 0.9250814332247557},
 1: {'F1-Score': 0.9293966623876766,
  'AUC-PR': 0.9758698759347404,
  'Accuracy': 0.9402823018458197},
 2: {'F1-Score': 0.9340232858990944,
  'AUC-PR': 0.9757153213406631,
  'Accuracy': 0.9446254071661238},
 3: {'F1-Score': 0.9387755102040817,
  'AUC-PR': 0.9795668851614707,
  'Accuracy': 0.9478827361563518},
 4: {'F1-Score': 0.9387755102040817,
  'AUC-PR': 0.9796881186326951,
  'Accuracy': 0.9478827361563518},
 5: {'F1-Score': 0.9293966623876766,
  'AUC-PR': 0.9758698759347404,
  'Accuracy': 0.9402823018458197},
 6: {'F1-Score': 0.9352331606217616,
  'AUC-PR': 0.9813380626444366,
  'Accuracy': 0.9457111834961998},
 7: {'F1-Score': 0.9352331606217616,
  'AUC-PR': 0.9813341873888645,
  'Accuracy': 0.9457111834961998},
 8: {'F1-Score': 0.9417596034696407,
  'AUC-PR': 0.9833251110322214,
  'Accuracy': 0.9489685124864278},
 9: {'F1-Score': 0.9448010269576379,
  'AUC-PR': 0.9840776825669

In [602]:
simple_to_complex01 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True)
}

simple_to_complex02 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "RandomForest_20trees": H2ORandomForestEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex04= {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "RandomForest_20trees": H2ORandomForestEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "RandomForest_50trees": H2ORandomForestEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex05 = {                   
   "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "RandomForest_50trees": H2ORandomForestEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex06 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}


simple_to_complex07 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees": H2ORandomForestEstimator(
                    ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                ),   
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex08 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex09 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_20trees_unbounded_D": H2ORandomForestEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex10 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "RandomForest_10trees_unbounded_D": H2ORandomForestEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "RandomForest_20trees_unbounded_D": H2ORandomForestEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "RandomForest_50trees_unbounded_D": H2ORandomForestEstimator(
    ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

random_forest_stacks_nbayes = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [604]:
comparative_results_rf_nbayes = dict()

for i, stack in enumerate(random_forest_stacks_nbayes):
    comparative_results_rf_nbayes[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training NaiveBayes with 5-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    NaiveBayes - F1-Score: 0.8303, AUC-PR: 0.8182, Accuracy (Test Set): 0.8588

>>> Metalearner's results:

    Super Learner - F1-Score: 0.8303, AUC-PR: 0.8156, Accuracy: 0.8588

>>> Training base learners:

    Training NaiveBayes with 5-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%
    Training RandomForest_10trees with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners

In [606]:
comparative_results_rf_nbayes

{0: {'F1-Score': 0.8302872062663186,
  'AUC-PR': 0.8156212228909406,
  'Accuracy': 0.8588490770901195},
 1: {'F1-Score': 0.9272503082614056,
  'AUC-PR': 0.9717393268505077,
  'Accuracy': 0.9370249728555917},
 2: {'F1-Score': 0.9323308270676691,
  'AUC-PR': 0.9735836638851284,
  'Accuracy': 0.9413680781758957},
 3: {'F1-Score': 0.9373368146214098,
  'AUC-PR': 0.9801066804200346,
  'Accuracy': 0.9478827361563518},
 4: {'F1-Score': 0.9384010484927915,
  'AUC-PR': 0.9802906334378295,
  'Accuracy': 0.9489685124864278},
 5: {'F1-Score': 0.9272503082614056,
  'AUC-PR': 0.9717393268505077,
  'Accuracy': 0.9370249728555917},
 6: {'F1-Score': 0.9341935483870969,
  'AUC-PR': 0.9721179310626975,
  'Accuracy': 0.9446254071661238},
 7: {'F1-Score': 0.9338521400778211,
  'AUC-PR': 0.9781291721202493,
  'Accuracy': 0.9446254071661238},
 8: {'F1-Score': 0.941025641025641,
  'AUC-PR': 0.9834072699744336,
  'Accuracy': 0.9500542888165038},
 9: {'F1-Score': 0.9455958549222797,
  'AUC-PR': 0.98486394167777

In [608]:
simple_to_complex01 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}

simple_to_complex02 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex04= {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex05 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex06 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}


simple_to_complex07 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                    ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                ),   
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex08 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex09 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex10 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

gradient_boosting_stacks = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [610]:
comparative_results_gb = dict()

for i, stack in enumerate(gradient_boosting_stacks):
    comparative_results_gb[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    LogisticRegression_binomial - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy (Test Set): 0.9251

>>> Metalearner's results:

    Super Learner - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy: 0.9251

>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training GradientBoosting_10trees with 5-fold cross-validation...
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |███████████████████

In [611]:
comparative_results_gb

{0: {'F1-Score': 0.9125475285171102,
  'AUC-PR': 0.9597218422898272,
  'Accuracy': 0.9250814332247557},
 1: {'F1-Score': 0.931758530183727,
  'AUC-PR': 0.9781269287795011,
  'Accuracy': 0.9435396308360477},
 2: {'F1-Score': 0.9404145077720208,
  'AUC-PR': 0.982245293612598,
  'Accuracy': 0.9500542888165038},
 3: {'F1-Score': 0.9491094147582697,
  'AUC-PR': 0.985633926339467,
  'Accuracy': 0.9565689467969598},
 4: {'F1-Score': 0.9491094147582697,
  'AUC-PR': 0.9857138714775479,
  'Accuracy': 0.9565689467969598},
 5: {'F1-Score': 0.931758530183727,
  'AUC-PR': 0.9781269287795011,
  'Accuracy': 0.9435396308360477},
 6: {'F1-Score': 0.931758530183727,
  'AUC-PR': 0.9783592567374547,
  'Accuracy': 0.9435396308360477},
 7: {'F1-Score': 0.9302325581395349,
  'AUC-PR': 0.9766319390181104,
  'Accuracy': 0.9413680781758957},
 8: {'F1-Score': 0.9346991037131882,
  'AUC-PR': 0.9813076643772469,
  'Accuracy': 0.9446254071661238},
 9: {'F1-Score': 0.9493670886075949,
  'AUC-PR': 0.9710341932881718,


In [612]:
simple_to_complex01 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
}

simple_to_complex02 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex04= {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex05 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex06 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}


simple_to_complex07 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                    ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                ),   
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex08 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex09 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex10 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

gradient_boosting_stacks_nbayes = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [613]:
comparative_results_gb_nbayes = dict()

for i, stack in enumerate(gradient_boosting_stacks_nbayes):
    comparative_results_gb_nbayes[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training NaiveBayes with 5-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    NaiveBayes - F1-Score: 0.8303, AUC-PR: 0.8182, Accuracy (Test Set): 0.8588

>>> Metalearner's results:

    Super Learner - F1-Score: 0.8303, AUC-PR: 0.8156, Accuracy: 0.8588

>>> Training base learners:

    Training NaiveBayes with 5-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%
    Training GradientBoosting_10trees with 5-fold cross-validation...
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base lear

In [629]:
simple_to_complex01 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}

simple_to_complex02 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}

simple_to_complex04= {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}


simple_to_complex05 = {                   
        "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32": H2ODeepLearningEstimator(
                        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}


simple_to_complex06 = {                   
        "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32": H2ODeepLearningEstimator(
                        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}


simple_to_complex07 = {            
        "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
                    hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
}


simple_to_complex08 = {         
        "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
                    hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
}

simple_to_complex09 = {       
        "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32": H2ODeepLearningEstimator(
                        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
                    hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),

}

simple_to_complex10 = {       
        "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32_32": H2ODeepLearningEstimator(
                        hidden=[32, 32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
}

neural_net_stacks = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [631]:
comparative_results_nns = dict()

for i, stack in enumerate(neural_net_stacks):
    comparative_results_nns[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    LogisticRegression_binomial - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy (Test Set): 0.9251

>>> Metalearner's results:

    Super Learner - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy: 0.9251

>>> Training base learners:

    Training LogisticRegression_binomial with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training NeuralNetwork_6 with 5-fold cross-validation...
deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |████████████████████████████

In [632]:
comparative_results_nns

{0: {'F1-Score': 0.9125475285171102,
  'AUC-PR': 0.9597218422898272,
  'Accuracy': 0.9250814332247557},
 1: {'F1-Score': 0.929471032745592,
  'AUC-PR': 0.9706865387483506,
  'Accuracy': 0.9391965255157437},
 2: {'F1-Score': 0.9385194479297365,
  'AUC-PR': 0.975450983779792,
  'Accuracy': 0.9467969598262758},
 3: {'F1-Score': 0.9366459627329192,
  'AUC-PR': 0.9767069820621652,
  'Accuracy': 0.9457111834961998},
 4: {'F1-Score': 0.9411764705882353,
  'AUC-PR': 0.9756518323050383,
  'Accuracy': 0.9500542888165038},
 5: {'F1-Score': 0.9444444444444444,
  'AUC-PR': 0.9788064971632069,
  'Accuracy': 0.9522258414766558},
 6: {'F1-Score': 0.9319371727748691,
  'AUC-PR': 0.9753691548470975,
  'Accuracy': 0.9435396308360477},
 7: {'F1-Score': 0.9331585845347313,
  'AUC-PR': 0.9781172006621045,
  'Accuracy': 0.9446254071661238},
 8: {'F1-Score': 0.9370277078085643,
  'AUC-PR': 0.9784595400137599,
  'Accuracy': 0.9457111834961998},
 9: {'F1-Score': 0.9372599231754161,
  'AUC-PR': 0.974241852770546

In [633]:
simple_to_complex01 = {                   
   "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
}

simple_to_complex02 = {                   
   "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
   "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}

simple_to_complex04= {                   
   "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}


simple_to_complex05 = {
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_32": H2ODeepLearningEstimator(
                        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}


simple_to_complex06 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32": H2ODeepLearningEstimator(
                        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}


simple_to_complex07 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
                    hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
}


simple_to_complex08 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
                    hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
}

simple_to_complex09 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_6": H2ODeepLearningEstimator(
                        hidden=[6], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_16": H2ODeepLearningEstimator(
                        hidden=[16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32": H2ODeepLearningEstimator(
                        hidden=[32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
                    "NeuralNetwork_32_16": H2ODeepLearningEstimator(
                    hidden=[32, 16], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),

}

simple_to_complex10 = {              
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
                    "NeuralNetwork_32_32": H2ODeepLearningEstimator(
                        hidden=[32, 32], epochs=300, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),
}

neural_net_stacks_nbayes = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [634]:
comparative_results_nns_nbayes = dict()

for i, stack in enumerate(neural_net_stacks_nbayes):
    comparative_results_nns_nbayes[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_test, X_train)


>>> Training base learners:

    Training NaiveBayes with 5-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    NaiveBayes - F1-Score: 0.8303, AUC-PR: 0.8182, Accuracy (Test Set): 0.8588

>>> Metalearner's results:

    Super Learner - F1-Score: 0.8303, AUC-PR: 0.8156, Accuracy: 0.8588

>>> Training base learners:

    Training NaiveBayes with 5-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%
    Training NeuralNetwork_6 with 5-fold cross-validation...
deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' res

# BALANCE CLASSES TRUE

## SIMPLE TO COMPLEX

To try all possible combinations would be computationally unfeaseble. Therefore we chose a more naive tactic, where we examine the combination of a 
simple learner with the a few representative selections of a single class of complex learners and we choose the best performing options for combinations with others.

In [571]:
from copy import deepcopy

def simple_copy_dict(dct):
    new_dct = dict()
    for elem in dct.items():
        new_dct[elem[0]] = elem[1] 
    return new_dct

def simpler_to_complex():
    base_learner_stacks_logistic_reg = [{list(simple_learners.items())[0][0]: list(simple_learners.items())[0][1]}]
    for random_forest in random_forests.items():
        base_learner_stacks_logistic_reg_snapshot = deepcopy(base_learner_stacks_logistic_reg)
        for stack in base_learner_stacks_logistic_reg_snapshot:
            new_stack = simple_copy_dict(stack)
            new_stack[random_forest[0]] = random_forest[1]
            base_learner_stacks_logistic_reg.append(new_stack)
            for gradient_boosting in gradient_boostings.items():
                base_learner_stacks_logistic_reg_snapshot = deepcopy(base_learner_stacks_logistic_reg)
                for stack in base_learner_stacks_logistic_reg_snapshot:
                    new_stack = simple_copy_dict(stack)
                    new_stack[gradient_boosting[0]] = gradient_boosting[1]
                    base_learner_stacks_logistic_reg.append(new_stack)
                    for neural_network in neural_networks.items():
                        base_learner_stacks_logistic_reg_snapshot = deepcopy(base_learner_stacks_logistic_reg)
                        for stack in base_learner_stacks_logistic_reg_snapshot:
                            new_stack = simple_copy_dict(stack)
                            new_stack[neural_network[0]] = neural_network[1]
                            base_learner_stacks_logistic_reg.append(new_stack)


    # base_learner_stacks_naive_bayes = [{list(simple_learners.items())[1][0]: list(simple_learners.items())[1][1]}]
    # for random_forest in random_forests.items():
    #     for stack in base_learner_stacks_naive_bayes:
    #         stack[random_forest[0]] = random_forest[1]
    #         for gradient_boosting in gradient_boostings.items():
    #             for stack in base_learner_stacks_naive_bayes:
    #                 stack[gradient_boosting[0]] = gradient_boosting[1]
    #                 for neural_network in neural_networks.items():
    #                     for stack in base_learner_stacks_naive_bayes:
    #                         stack[neural_network[0]] = neural_network[1]

    # base_learner_stacks_naive_bayes_logistic_reg = [simple_learners]
    # for random_forest in random_forests.items():
    #     for stack in base_learner_stacks_naive_bayes_logistic_reg:
    #         stack[random_forest[0]] = random_forest[1]
    #         for gradient_boosting in gradient_boostings.items():
    #             for stack in base_learner_stacks_naive_bayes_logistic_reg:
    #                 stack[gradient_boosting[0]] = gradient_boosting[1]
    #                 for neural_network in neural_networks.items():
    #                     for stack in base_learner_stacks_naive_bayes_logistic_reg:
    #                         stack[neural_network[0]] = neural_network[1]
    return base_learner_stacks_logistic_reg#, base_learner_stacks_naive_bayes, base_learner_stacks_naive_bayes_logistic_reg
                        

KeyboardInterrupt: 

## TRYING OUT GRID SEARCH FROM
https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/stacked-ensembles.html

In [10]:
from h2o.grid.grid_search import H2OGridSearch

# Specify GBM hyperparameters for the grid
hyper_params = {"learn_rate": [0.01, 0.03],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 3, "seed": 1}

# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees=10,
                                                        seed=1,
                                                        nfolds=N_FOLDS,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
grid.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

# Train a stacked ensemble using the GBM grid
super_learner = H2OStackedEnsembleEstimator(model_id="my_ensemble_gbm_grid_binomial",
                                       base_models=grid.model_ids)
super_learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)



super_performance = super_learner.model_performance(test_data=h2o_test)
super_accuracy = super_performance.accuracy()[0][1]
super_f1 = super_performance.F1()[0][1]  
super_auc_pr = super_performance.aucpr()  
print(f"\nSuper Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f} | Super Learner Accuracy: {super_accuracy:.4f}")


print("\nFinal Results Comparison:")
for name, metrics in results.items():
    print(f"{name} - F1-Score: {metrics['F1-Score']:.4f}, AUC-PR: {metrics['AUC-PR']:.4f}, Accuracy: {metrics['Accuracy']:.4f}")
    
print(f"Super Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f}, Accuracy: {super_accuracy:.4f}")



gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

Super Learner - F1-Score: 0.9096, AUC-PR: 0.9690 | Super Learner Accuracy: 0.9240

Final Results Comparison:
LogisticRegression - F1-Score: 0.9125, AUC-PR: 0.9597, Accuracy: 0.9251
RandomForest - F1-Score: 0.9375, AUC-PR: 0.9790, Accuracy: 0.9479
GradientBoosting - F1-Score: 0.9490, AUC-PR: 0.9848, Accuracy: 0.9566
NaiveBayes - F1-Score: 0.8303, AUC-PR: 0.8182, Accuracy: 0.8588
Super Learner - F1-Score: 0.9096, AUC-PR: 0.9690, Accuracy: 0.9240
